In [1]:
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.utils.DimensionAwareStrategy import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 
from pyMSOO.utils.MultiRun.RunMultiBenchmark import * 

In [2]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(1)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()

In [9]:
import numpy as np
import random
from numba import jit, njit

from pyMSOO.MFEA.model import AbstractModel
from pyMSOO.utils import Crossover, Mutation, Selection, DimensionAwareStrategy
from pyMSOO.utils.EA import *
from pyMSOO.utils.numba_utils import numba_randomchoice, numba_random_gauss, numba_random_cauchy, numba_random_uniform
from pyMSOO.utils.Search import *

class model(AbstractModel.model):
    TOLERANCE = 1e-6
    INF = 1e8
    def compile(self, 
        IndClass: Type[Individual],
        tasks: List[AbstractTask], 
        crossover: Crossover.SBX_Crossover, 
        mutation: Mutation.PolynomialMutation, 
        selection: Selection.ElitismSelection,
        dimension_strategy: DimensionAwareStrategy.AbstractDaS = DimensionAwareStrategy.NoDaS(),
        *args, **kwargs):
        super().compile(IndClass, tasks, crossover, mutation,dimension_strategy, selection, *args, **kwargs)
    
    def fit(self, nb_generations, 
            nb_inds_each_task = 100, 
            nb_inds_max = 100,
            nb_inds_min = 20,
            evaluate_initial_skillFactor = True, 
            c = 0.06,
            *args, 
            **kwargs) -> List[Individual]:
        super().fit(*args, **kwargs)

        # nb_inds_min
        if nb_inds_min is not None:
            assert nb_inds_each_task >= nb_inds_min
        else: 
            nb_inds_min = nb_inds_each_task

        self.rmp = np.full((len(self.tasks), len(self.tasks)), 0.3)
        self.learningPhase = [LearningPhase(self.IndClass, self.tasks, t) for t in self.tasks]
        
        # initialize population
        self.population = Population(
            self.IndClass,
            nb_inds_tasks = [nb_inds_each_task] * len(self.tasks), 
            dim = self.dim_uss,
            list_tasks= self.tasks,
            evaluate_initial_skillFactor = evaluate_initial_skillFactor
        )

        self.nb_inds_tasks = [nb_inds_each_task] * len(self.tasks)

        MAXEVALS = nb_generations * nb_inds_each_task * len(self.tasks)
        self.max_eval_k = [nb_generations * nb_inds_each_task] * len(self.tasks)
        self.eval_k = [0] * len(self.tasks)
        epoch = 1
        
        D0 = self.calculateD(population = np.array([[ind.genes for ind in sub.ls_inds] for sub in self.population]), 
                            population_fitness = np.array([sub.getFitness() for sub in self.population]),
                            best = np.array([sub.__getBestIndividual__.genes for sub in self.population]),)

        while sum(self.eval_k) < MAXEVALS:
            self.delta = [[[] for _ in range(len(self.tasks))] for _ in range(len(self.tasks))]

            self.s_rmp = [[[] for _ in range(len(self.tasks))] for _ in range(len(self.tasks))]

            self.population.update_rank()
            # print(self.eval_k)
            if sum(self.eval_k) >= epoch * nb_inds_each_task * len(self.tasks):
                # save history
                self.history_cost.append([ind.fcost for ind in self.population.get_solves()])
                self.render_process(epoch/nb_generations, ['Pop_size', 'Cost'], [[sum(self.nb_inds_tasks)], self.history_cost[-1]], use_sys= True)
                epoch += 1

            # offsprings = self.reproduction(sum(self.nb_inds_tasks), self.population)

            # self.population = self.population + offsprings
            # start = time.time()
            matingPool = Population(
                self.IndClass,
                nb_inds_tasks = [0] * len(self.tasks), 
                dim = self.dim_uss,
                list_tasks= self.tasks,
                evaluate_initial_skillFactor = False
            )

            for idx in range(len(self.tasks)):
                
                idx_inds = np.argsort([ind.fcost for ind in self.population[idx].ls_inds])
                
                for i in idx_inds[:int(len(self.population[idx])/2)]:
                    matingPool.__addIndividual__(self.population[idx].ls_inds[i])

            offsprings = self.reproduction(len(self.population), matingPool)
        
            # # merge and update rank
            self.population = matingPool + offsprings
            self.population.update_rank()
            # end = time.time()
            # print("E: ", end - start)
            # selection
            self.nb_inds_tasks = [int(
                int(max((nb_inds_min - nb_inds_max) * (sum(self.eval_k)/MAXEVALS) + nb_inds_max, nb_inds_min))
            )] * len(self.tasks)
            self.selection(self.population, self.nb_inds_tasks)

            # update operators
            self.crossover.update(population = self.population)
            self.mutation.update(population = self.population)
            self.dimension_strategy.update(population = self.population)
            # start = time.time()
            self.updateRMP(c)
            # end = time.time()
            # print("G: ", end - start)
            # start = time.time()
            self.phaseTwo(D0)
            # end = time.time()
            # print("G: ", end - start)
        # self.phaseTwo(D0)
        print('\nEND!')

        #solve 
        self.render_process(epoch/nb_generations, ['Pop_size', 'Cost'], [[sum(self.nb_inds_tasks)], self.history_cost[-1]], use_sys= True)
        self.population.update_rank()
        self.last_pop = self.population
        return self.last_pop.get_solves()
    
    def reproduction(self, size: int, mating_pool: Population,) -> Population:
        sub_size = int(size/len(self.tasks))
       
        offsprings = Population(self.IndClass,
                                nb_inds_tasks = [0] * len(self.tasks), 
                                dim = self.dim_uss,
                                list_tasks= self.tasks)
        counter = np.zeros((len(self.tasks)))  

        stopping = False
        while not stopping:
            pa, pb = mating_pool.__getRandomInds__(2)
            ta = pa.skill_factor
            tb = pb.skill_factor

            if counter[ta] >= sub_size and counter[tb] >= sub_size:
                continue

            rmpValue = numba_random_gauss(mean = max(self.rmp[ta][tb], self.rmp[tb][ta]), sigma = 0.1)

            if ta == tb:
                # self.eval_k[ta] += 2

                oa, ob = self.crossover(pa, pb)

                oa.skill_factor = ta
                ob.skill_factor = ta

                if self.eval_k[ta] >= self.max_eval_k[ta]:
                    oa.fcost = model.INF
                else:
                    self.eval_k[ta] += 1

                offsprings.__addIndividual__(oa)

                if self.eval_k[tb] >= self.max_eval_k[tb]:
                    ob.fcost = model.INF
                else:
                    self.eval_k[tb] += 1

                offsprings.__addIndividual__(ob)

                counter[ta] += 2

            elif random.random() <= rmpValue:
                off = self.crossover(pa, pb)

                for o in off:
                    if counter[ta] < sub_size and random.random() < self.rmp[ta][tb]/(self.rmp[ta][tb] + self.rmp[tb][ta]):
                        o.skill_factor = ta
                        o = self.dimension_strategy(o, tb, pa)
                        if self.eval_k[ta] >= self.max_eval_k[ta]:
                            o.fcost = model.INF
                        else:
                            self.eval_k[ta] += 1
                            o.fcost = self.tasks[ta](o)

                        offsprings.__addIndividual__(o)
                        
                        counter[ta] += 1
                        # self.eval_k[ta] += 1
                        
                        if pa.fcost > o.fcost:
                            self.delta[ta][tb].append(pa.fcost - o.fcost)
                            self.s_rmp[ta][tb].append(rmpValue)
                    
                    elif counter[tb] < sub_size:
                        o.skill_factor = tb
                        o = self.dimension_strategy(o, ta, pb)
        
                        if self.eval_k[tb] >= self.max_eval_k[tb]:
                            o.fcost = model.INF
                        else:
                            self.eval_k[tb] += 1
                            o.fcost = self.tasks[tb](o)

                        offsprings.__addIndividual__(o)
                        
                        counter[tb] += 1
                        # self.eval_k[tb] += 1

                        if pb.fcost > o.fcost:
                            self.delta[tb][ta].append(pb.fcost - o.fcost)
                            self.s_rmp[tb][ta].append(rmpValue)

            else:
                if counter[ta] < sub_size:
                    paa: Individual = self.population[ta].__getRandomItems__()

                    # while np.array_equal(paa.genes, pa.genes):
                    #     paa: Individual = self.population[ta].__getRandomItems__()
                    
                    oa, _ = self.crossover(pa, paa)
                    oa.skill_factor = ta
                    
                    if self.eval_k[ta] >= self.max_eval_k[ta]:
                        oa.fcost = model.INF
                    else:
                        self.eval_k[ta] += 1
                        oa.fcost = self.tasks[ta](oa)

                    offsprings.__addIndividual__(oa)

                    counter[ta] += 1
                    # self.eval_k[ta] += 1

                if counter[tb] < sub_size:
                    pbb: Individual = self.population[tb].__getRandomItems__()

                    # while np.array_equal(pbb.genes, pb.genes):
                    #     pbb: Individual = self.population[tb].__getRandomItems__()
                    
                    ob, _ = self.crossover(pb, pbb)
                    ob.skill_factor = tb

                    if self.eval_k[tb] >= self.max_eval_k[tb]:
                        ob.fcost = model.INF
                    else:
                        self.eval_k[tb] += 1
                        ob.fcost = self.tasks[tb](ob)

                    offsprings.__addIndividual__(ob)
                    
                    counter[tb] += 1
                    # self.eval_k[tb] += 1
                    
            stopping = sum(counter >= sub_size) == len(self.tasks)

        return offsprings

    def phaseTwo(self, D0):
        fcosts = [sub.getFitness() for sub in self.population]
        # start = time.time()
        D = self.calculateD(population = np.array([[ind.genes for ind in sub.ls_inds]for sub in self.population]), 
                            population_fitness = np.array(fcosts),
                            best = np.array([sub.__getBestIndividual__.genes for sub in self.population]),
                            )
        # end = time.time()
        # print("A: ", end - start)
        maxFit = np.max(fcosts, axis=1)
        minFit = np.min(fcosts, axis=1)
        maxDelta = maxFit - minFit + 1e-99

        assert len(D) == len(maxDelta), "Wrong shape. Got {} and {}".format(D.shape, maxDelta.shape)
        assert len(D) == len(self.tasks), "Got wrong shape"

        sigma = np.where(D > D0, 0, 1 - D/D0)
        nextPop = Population(IndClass = self.IndClass,
                            dim = self.dim_uss,
                            nb_inds_tasks=[0] * len(self.tasks),
                            list_tasks=self.tasks)
        # start = time.time()
        for i in range(len(self.tasks)):
            self.eval_k[i] += self.learningPhase[i].evolve(self.max_eval_k[i] - self.eval_k[i], 
                                                           self.population[i], 
                                                           nextPop, 
                                                           sigma[i], 
                                                           maxDelta[i])
        # end = time.time()
        # print("B: ", end - start)
        self.population = nextPop

    def calculateD(self, population: np.array, population_fitness: np.array, best: np.array) -> np.array:
        '''
        Arguments include:\n
        + `population`: genes of the current population
        + `population_fitness`: fitness of the current population
        + `best`: the best gene of each subpop
        + `nb_tasks`: number of tasks
        '''
        
        D = np.empty((len(self.tasks)))
        for i in range(len(self.tasks)):
            gene_max = [np.max(population[i], axis = 1).tolist()] * self.dim_uss
            gene_min = [np.min(population[i], axis = 1).tolist()] * self.dim_uss

            D[i] = self.__class__._calculateD(np.array(gene_max).T, np.array(gene_min).T, population[i], population_fitness[i], best[i], model.TOLERANCE)
        return D
    
    @jit(nopython = True, parallel = True, cache=True)
    def _calculateD(gene_max: np.array, gene_min: np.array, subPop: np.array, subPop_fitness: np.array, best: np.array, TOLERANCE: float) -> float:
            # gene_max = gene_max.flatten()
            # gene_max = np.broadcast_to(gene_max, (subPop.shape[-1], subPop.shape[0])).T
            # gene_min = gene_min.flatten()
            # gene_min = np.broadcast_to(gene_min, (subPop.shape[-1], subPop.shape[0])).T
            
            w = np.where(subPop_fitness > TOLERANCE, 1/(subPop_fitness), 1/TOLERANCE)
            # w = [1/ind if ind > TOLERANCE else 1/TOLERANCE for ind in population[i]]
            # print(subPop.shape)
            sum_w = sum(w)
            d = (subPop - gene_min)/(gene_max - gene_min)
            best = (best - gene_min)/(gene_max - gene_min)
            d = np.sum((d - best) ** 2, axis=1)
            d = np.sqrt(d)
            assert d.shape == w.shape
            # d = np.sqrt(np.sum(d, axis=0))
            # d = np.sum([np.sqrt(np.sum((d[i] - best) * (d[i] - best))) for i in range(len(subPop))])

            return np.sum(w * d/sum_w)
    
    def updateRMP(self, c: int):
        for i in range(len(self.tasks)):
            for j in range(len(self.tasks)):
                if i == j:
                    continue
                if len(self.delta[i][j]) > 0:
                    self.rmp[i][j] += self.__class__._updateRMP(self.delta[i][j], self.s_rmp[i][j], c)
                else:
                    self.rmp[i][j] = (1 - c) * self.rmp[i][j]
                
                self.rmp[i][j] = max(0.1, min(1, self.rmp[i][j]))

    @jit(nopython = True, parallel = True, cache= True)
    def _updateRMP(delta: List, s_rmp: List, c: float) -> float:
        delta = np.array(delta)
        s_rmp = np.array(s_rmp)
        sum_delta = sum(delta)
        tmp = (delta/sum_delta) * s_rmp
        meanS = sum(tmp * s_rmp)
        
        return c * meanS/sum(tmp)
    
class LearningPhase():
    M = 2
    H = 10
    def __init__(self, IndClass, list_tasks, task) -> None:
        self.IndClass = IndClass
        self.list_tasks = list_tasks
        self.task = task
        self.sum_improv = [0.0] * LearningPhase.M
        self.consume_fes = [1.0] * LearningPhase.M
        self.mem_cr = [0.5] * LearningPhase.H
        self.mem_f = [0.5] * LearningPhase.H
        self.s_cr = []
        self.s_f = []
        self.diff_f = []
        self.mem_pos = 0
        self.gen = 0
        self.best_opcode = 1
        self.searcher = [self.pbest1, PolynomialMutation(nm = 5).getInforTasks(self.IndClass, self.list_tasks)]

    def evolve(self, max_eval: int, subPop: SubPopulation, nextPop: Population, sigma: float, max_delta: float) -> SubPopulation:
        self.gen += 1

        if self.gen > 1:
            # start = time.time()
            self.best_opcode = self.__class__.updateOperator(sum_improve = self.sum_improv, 
                                                             consume_fes = self.consume_fes, 
                                                             M = LearningPhase.M)

            self.sum_improv = [0.0] * LearningPhase.M
            self.consume_fes = [1.0] * LearningPhase.M

            # end = time.time()
            # print("C: ", end - start)

        # self.updateMemory()
        
        pbest_size = max(5, int(0.15 * len(subPop)))
        # pbest = subPop.__getRandomItems__(size = pbest_size)
        idx_inds = np.argsort([ind.fcost for ind in subPop.ls_inds])
        pbest =  [subPop.ls_inds[i] for i in idx_inds[:pbest_size]] 
        # start1 = time.time()
        for i, ind in enumerate(subPop):
            # start1 = time.time()
            # start = time.time()
            r = random.randint(0, LearningPhase.M - 1)
            cr = numba_random_gauss(self.mem_cr[r], 0.1)
            f = numba_random_cauchy(self.mem_f[r], 0.1)
            # end = time.time()
            # print("A: ", end - start)
            opcode = random.randint(0, LearningPhase.M)
            if opcode == LearningPhase.M:
                opcode = self.best_opcode

            self.consume_fes[opcode] += 1
            
            if opcode == 0:
                # start = time.time()
                child = self.searcher[opcode](ind, subPop, pbest, cr, f)
                # end = time.time()
                # print("C: ", end - start)
            elif opcode == 1:
                # start = time.time()
                child = self.searcher[opcode](ind, return_newInd=True)
                # end = time.time()
                # print("D: ", end - start)

            # start = time.time()
            child.skill_factor = ind.skill_factor
            if i <= max_eval:
                child.fcost = self.task(child)
            else:
                child.fcost = model.INF
            
            diff = ind.fcost - child.fcost
            if diff > 0:
                survival = child

                self.sum_improv[opcode] += diff

                if opcode == 0:
                    self.diff_f.append(diff)
                    # self.s_cr.append(cr)
                    # self.s_f.append(f)
                
            elif diff == 0 or random.random() <= sigma * np.exp(diff/max_delta):
                survival = child
            else:
                survival = ind
            
            nextPop.__addIndividual__(survival)
            # end = time.time()
            # print("M: ", end - start)
        # end = time.time()
        # print("F: ", end - start1)
        return min(len(subPop), max_eval)
    
    def pbest1(self, ind: Individual, subPop: SubPopulation, best: List[Individual], cr: float, f: float) -> Individual:
        pbest = best[random.randint(0, len(best) - 1)]
        
        ind_ran1, ind_ran2 = subPop.__getRandomItems__(size = 2, replace= False)
        
        u = (numba_random_uniform(len(ind.genes)) < cr)
        if np.sum(u) == 0:
            u = np.zeros(shape= (subPop.dim,))
            u[numba_randomchoice(subPop.dim)] = 1

        # new_genes = np.where(u, 
        #     pbest.genes + f * (ind_ran1.genes - ind_ran2.genes),
        #     ind.genes
        # )
        # # new_genes = np.clip(new_genes, ind.genes/2, (ind.genes + 1)/2)
        # new_genes = np.where(new_genes < 0, ind.genes/2, np.where(new_genes > 1, (ind.genes + 1)/2, new_genes))

        new_genes = self.__class__.produce_inds(ind.genes, pbest.genes, ind_ran1.genes, ind_ran2.genes, f, u)
        new_ind = self.IndClass(new_genes)

        return new_ind

    @jit(nopython=True, parallel = True)
    def produce_inds(ind_genes: np.array, best_genes: np.array, ind1_genes: np.array, ind2_genes: np.array, F: float, u: np.array) -> np.array:
        new_genes = np.where(u,
            best_genes + F * (ind1_genes - ind2_genes),
            ind_genes
        )
        new_genes = np.where(new_genes > 1, (ind_genes + 1)/2, new_genes) 
        new_genes = np.where(new_genes < 0, (ind_genes + 0)/2, new_genes)

        return new_genes

    # def updateMemory(self):
    #     if len(self.s_cr) > 0:
    #         # self.diff_f = np.array(self.diff_f)
    #         # self.s_cr = np.array(self.s_cr)
    #         # self.s_f = np.array(self.s_f)

    #         self.mem_cr[self.mem_pos] = self.__class__.updateMemoryCR(self.diff_f, self.s_cr)
    #         self.mem_f[self.mem_pos] = self.__class__.updateMemoryF(self.diff_f, self.s_f)
            
    #         self.mem_pos = (self.mem_pos + 1) % LearningPhase.H

    #         self.s_cr = []
    #         self.s_f = []
    #         self.diff_f = []

    # @jit(nopython = True, parallel = True, cache=True)
    # def updateMemoryCR(diff_f: List, s_cr: List) -> float:
    #     diff_f = np.array(diff_f)
    #     s_cr = np.array(s_cr)

    #     sum_diff = sum(diff_f)
    #     weight = diff_f/sum_diff
    #     tmp_sum_cr = sum(weight * s_cr)
    #     mem_cr = sum(weight * s_cr * s_cr)
        
    #     if tmp_sum_cr == 0 or mem_cr == -1:
    #         return -1
    #     else:
    #         return mem_cr/tmp_sum_cr
        
    # @jit(nopython = True, parallel = True, cache = True)
    # def updateMemoryF(diff_f: List, s_f: List) -> float:
    #     diff_f = np.array(diff_f)
    #     s_f = np.array(s_f)

    #     sum_diff = sum(diff_f)
    #     weight = diff_f/sum_diff
    #     tmp_sum_f = sum(weight * s_f)
    #     return sum(weight * (s_f ** 2)) / tmp_sum_f

    @jit(nopython = True, parallel = True)
    def updateOperator(sum_improve: List, consume_fes: List, M: int) -> int:
        sum_improve = np.array(sum_improve)
        consume_fes = np.array(consume_fes)
        eta = sum_improve / consume_fes
        best_rate = max(eta)
        best_op = np.argmax(eta)
        if best_rate > 0:
            return best_op
        else:
            return random.randint(0, M - 1)

In [10]:
baseModel = model()
# from pyMSOO.MFEA.model import EME_BI
# baseModel = EME_BI.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    # dimension_strategy= DimensionAwareStrategy.DaS_strategy()
)
solve = baseModel.fit(
    nb_generations = 1000, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

f:\Anaconda\Anaconda\envs\MFEA\lib\site-packages\numba\core\ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'delta' of function 'model._updateRMP'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "C:\Users\Acer\AppData\Local\Temp\ipykernel_16676\2371826582.py", line 342:
    @jit(nopython = True, parallel = True, cache= True)
    def _updateRMP(delta: List, s_rmp: List, c: float) -> float:
    ^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
f:\Anaconda\Anaconda\envs\MFEA\lib\site-packages\numba\core\ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 's_rmp' of function 'model._updateRMP'.

For more information visit https://numba.readthedocs.io/en/stable/refe

Seed: None -- Time: 00m 8.90s    0 % [>                   ]  Pop_size: 9.90E+02  ,  Cost: 8.79E+04  2.52E+05  2.30E+05  2.92E+01  2.45E+09  2.13E+01  7.99E+01  1.64E+04  5.65E+01  5.82E+04  ,  

f:\Anaconda\Anaconda\envs\MFEA\lib\site-packages\numba\core\ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'consume_fes' of function 'LearningPhase.updateOperator'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "C:\Users\Acer\AppData\Local\Temp\ipykernel_16676\2371826582.py", line 521:
    @jit(nopython = True, parallel = True)
    def updateOperator(sum_improve: List, consume_fes: List, M: int) -> int:
    ^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
f:\Anaconda\Anaconda\envs\MFEA\lib\site-packages\numba\core\ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'sum_improve' of function 'LearningPhase.updateOperator'.

For more information visit https://

Seed: None -- Time: 03m 1.69s   99 % [===================>]  Pop_size: 2.00E+02  ,  Cost: 0.00E+00  1.29E-26  0.00E+00  0.00E+00  1.27E-25  4.31E-14  0.00E+00  6.36E-04  0.00E+00  2.59E+01  ,   
END!
Seed: None -- Time: 03m 3.37s  100 % [====================>]  Pop_size: 2.00E+02  ,  Cost: 0.00E+00  1.13E-26  0.00E+00  0.00E+00  1.06E-25  4.31E-14  0.00E+00  6.36E-04  0.00E+00  2.59E+01  ,  

In [11]:
len(baseModel.population)

200

In [13]:
baseModel.eval_k

[100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000,
 100000]

In [16]:
print([lp.best_opcode for lp in baseModel.learningPhase])

[0, 0, 0, 1, 0, 0, 1, 1, 1, 0]


In [38]:

baseModel = model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy= DaS_strategy()
)
solve = baseModel.fit(
    nb_generations = 1000, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Seed: None -- Time: 00m 1.96s    0 % [>                   ]  Pop_size: 9.90E+02  ,  Cost: 1.02E+05  2.32E+05  2.40E+05  3.47E+01  2.87E+09  2.13E+01  7.91E+01  1.56E+04  5.74E+01  6.42E+04  ,  

KeyboardInterrupt: 

In [17]:
import pandas as pd

df = pd.read_csv(r"F:\BTVN\DSAI\Optimization Lab\Paper\Efficient knowledge transfer\history_cost_summaries_EME_BI_woDaS.csv")

In [18]:
df.tail(50)

,Unnamed: 0,EME-BI,EME_BI_wo_DaS
450,450,0.000000,0.649250
451,451,153.862380,140.877084
452,452,0.000000,0.000329
453,453,10.890937,23.687499
454,454,5022.383910,6610.532774
455,455,0.000000,8.183991
456,456,112.875490,159.721696
457,457,0.000000,0.000000
458,458,12.390112,21.295734
459,459,4979.867545,6728.074941


In [41]:
# from pyMSOO.MFEA.model import EME_BI

# ls_benchmark = []
# ls_IndClass = []
# name_benchmark = []
# ls_tasks = [10]

# for i in ls_tasks:
#     # t, ic = WCCI22_benchmark.get_complex_benchmark(i)
#     t, ic = WCCI22_benchmark.get_50tasks_benchmark(i)
#     ls_benchmark.append(t)
#     ls_IndClass.append(ic)
#     name_benchmark.append(str(i))



# smpModel = MultiBenchmark(
#     ls_benchmark= ls_benchmark,
#     name_benchmark= name_benchmark,
#     ls_IndClass= ls_IndClass,
#     model= EME_BI
# )

# smpModel.compile(
#     crossover= SBX_Crossover(nc = 2),
#     mutation= PolynomialMutation(nm = 5),
#     selection= ElitismSelection(),
#     # dimension_strategy = DaS_strategy(eta = 3)
# )
# smpModel.fit(
#     nb_generations = 1000,nb_inds_each_task= 100, 
#     bound_pop= [0, 1], evaluate_initial_skillFactor= True
# )
# a = smpModel.run(
#     nb_run= 1)

In [33]:
res = []
# for i in range(1, 11):
t, ic = WCCI22_benchmark.get_50tasks_benchmark(9)
baseModel = model()
baseModel.compile(
    IndClass= ic,
    tasks= t,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    # dimension_strategy= DaS_strategy()
)
solve = baseModel.fit(
    nb_generations = 1000, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

# res.append([subpop.getSolveInd().fcost for subpop in baseModel.last_pop])

Seed: None -- Time: 17m 35.56s   99 % [===================>]  Pop_size: 1.00E+03  ,  Cost: 1.19E+02  2.10E-09  1.20E+02  1.30E-12  5.64E+00  5.12E+03  4.82E+01  1.50E-13  1.02E+02  1.32E-11  1.19E+00  7.38E+03  4.68E+01  5.67E-10  1.15E+02  2.40E-11  8.62E+00  5.41E+03  3.09E+02  2.00E+01  9.74E+01  2.33E-11  2.35E+00  7.70E+03  4.72E+01  2.87E-12  1.00E+02  6.92E-11  6.37E+00  6.32E+03  4.75E+01  7.57E-11  1.59E+02  2.75E-10  5.51E+00  6.43E+03  4.87E+01  2.00E+01  1.35E+02  9.94E-11  6.38E-01  5.05E+03  4.69E+01  3.89E-11  1.24E+02  1.14E-11  1.82E+00  5.44E+03  4.86E+01  2.01E+01  ,  
END!
Seed: None -- Time: 17m 46.01s  100 % [====================>]  Pop_size: 1.00E+03  ,  Cost: 1.19E+02  2.10E-09  1.19E+02  1.01E-12  5.64E+00  5.12E+03  4.82E+01  7.86E-14  1.01E+02  1.11E-11  1.19E+00  7.38E+03  4.68E+01  5.67E-10  1.14E+02  2.20E-11  8.62E+00  5.41E+03  3.09E+02  2.00E+01  8.93E+01  2.31E-11  2.35E+00  7.70E+03  4.72E+01  2.57E-12  9.23E+01  6.73E-11  6.37E+00  6.32E+03  4.75E+01

In [34]:
h = np.array([subpop.getSolveInd().fcost for subpop in baseModel.last_pop])

In [35]:
h = np.where(h < 1e-6, 0, h)

In [36]:
print(h)
print(np.array(df["EME-BI"][-100:-50]))

[1.19332709e+02 0.00000000e+00 1.18894573e+02 0.00000000e+00
 5.63568780e+00 5.12286024e+03 4.81605226e+01 0.00000000e+00
 1.01421384e+02 0.00000000e+00 1.18984303e+00 7.37960500e+03
 4.68279935e+01 0.00000000e+00 1.13625382e+02 0.00000000e+00
 8.61583492e+00 5.40961486e+03 3.08511018e+02 1.99848761e+01
 8.92349306e+01 0.00000000e+00 2.35393013e+00 7.70491790e+03
 4.72076715e+01 0.00000000e+00 9.22279775e+01 0.00000000e+00
 6.36707861e+00 6.31764802e+03 4.74683764e+01 0.00000000e+00
 1.40387984e+02 0.00000000e+00 5.50802877e+00 6.43279297e+03
 4.87159261e+01 1.99722336e+01 1.32008821e+02 0.00000000e+00
 6.38473375e-01 5.05431253e+03 4.68672370e+01 0.00000000e+00
 1.20272996e+02 0.00000000e+00 1.82315689e+00 5.43768031e+03
 4.86259257e+01 2.01062843e+01]
[1345.2809961     0.          136.0607488     0.           11.06903917
 4493.66155547  295.07901483    0.          132.30275237    0.
   12.30391307 4708.67148357  330.7125527     0.          135.2337131
    0.           10.8210762  491

In [59]:
sum(h > df["EME-BI"][-100:-50])

13

In [60]:
sum(h < df["EME-BI"][-100:-50])

23

In [41]:
res = []
# for i in range(1, 11):
t, ic = WCCI22_benchmark.get_50tasks_benchmark(9)
baseModel = model()
baseModel.compile(
    IndClass= ic,
    tasks= t,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy= DaS_strategy()
)
solve = baseModel.fit(
    nb_generations = 1000, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

# res.append([subpop.getSolveInd().fcost for subpop in baseModel.last_pop])

Seed: None -- Time: 17m 48.00s   99 % [===================>]  Pop_size: 1.00E+03  ,  Cost: 4.68E+01  2.00E+01  1.10E+02  1.54E-11  6.19E+00  5.77E+03  4.52E+01  2.74E-12  1.13E+02  4.55E-12  7.42E-01  7.80E+03  4.91E+01  8.32E-11  3.60E+02  5.27E-11  8.93E+00  5.69E+03  4.92E+01  9.21E-13  9.39E+01  4.23E-12  1.71E+01  6.13E+03  4.51E+01  2.00E+01  9.10E+01  1.14E-10  1.13E+00  6.45E+03  1.63E+02  2.00E+01  8.21E+01  3.18E-10  9.70E+00  5.61E+03  3.31E+02  1.81E-11  1.97E+02  1.12E-11  3.07E+00  3.77E+03  4.56E+01  2.00E+01  1.05E+02  2.96E-12  2.05E+00  6.29E+03  1.43E+02  1.60E-10  ,  
END!
Seed: None -- Time: 17m 58.19s  100 % [====================>]  Pop_size: 1.00E+03  ,  Cost: 4.68E+01  2.00E+01  1.03E+02  1.39E-11  6.19E+00  5.77E+03  4.52E+01  1.20E-12  1.13E+02  4.29E-12  7.42E-01  7.80E+03  4.91E+01  7.97E-11  3.28E+02  5.24E-11  8.93E+00  5.69E+03  4.92E+01  7.61E-13  8.97E+01  3.84E-12  1.71E+01  6.13E+03  4.51E+01  2.00E+01  8.64E+01  1.11E-10  1.13E+00  6.45E+03  1.63E+02

In [42]:
f = np.array([subpop.getSolveInd().fcost for subpop in baseModel.last_pop])

In [44]:
f = np.where(f < 1e-6, 0, f)

In [53]:
print(f)
print(np.array(df["EME-BI"][400:450]))

[4.68003042e+01 1.99663587e+01 1.03113956e+02 0.00000000e+00
 6.19448226e+00 5.76956100e+03 4.52351287e+01 0.00000000e+00
 1.12963484e+02 0.00000000e+00 7.41577009e-01 7.80272560e+03
 4.91021492e+01 0.00000000e+00 3.24258332e+02 0.00000000e+00
 8.92674103e+00 5.69427436e+03 4.91778275e+01 0.00000000e+00
 8.97199871e+01 0.00000000e+00 1.71331118e+01 6.12955815e+03
 4.51297589e+01 1.99613433e+01 8.63315608e+01 0.00000000e+00
 1.12805856e+00 6.45339352e+03 1.62864812e+02 1.99926680e+01
 8.19327505e+01 0.00000000e+00 9.70125321e+00 5.60552946e+03
 3.22637544e+02 0.00000000e+00 1.95727540e+02 0.00000000e+00
 3.06602228e+00 3.77352536e+03 4.56151967e+01 2.00148003e+01
 1.05114663e+02 0.00000000e+00 2.04655754e+00 6.29422535e+03
 1.43124317e+02 0.00000000e+00]
[1345.2809961     0.          136.0607488     0.           11.06903917
 4493.66155547  295.07901483    0.          132.30275237    0.
   12.30391307 4708.67148357  330.7125527     0.          135.2337131
    0.           10.8210762  491

In [54]:
sum(f < df["EME-BI"][400:450])

22

In [56]:
sum(f > df["EME-BI"][400:450])

15

In [57]:
sum(h < f)

23

In [58]:
sum(h > f)

17

In [23]:
sum(f < df["EME_BI_wo_DaS"][-50:])

34

In [62]:
sum(df["EME_BI_wo_DaS"][-50:] < df["EME-BI"][-50:])

2

In [52]:
sum(df["EME_BI_wo_DaS"][-50:] > df["EME-BI"][-50:])

43